In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
btc_return = pd.read_csv("../Data/BTC_return.csv", index_col=0, parse_dates=True)
btc_return

,Returns
2014-09-18,-0.074643
2014-09-19,-0.072402
2014-09-20,0.035111
2014-09-21,-0.024968
2014-09-22,0.008317
...,...
2024-03-28,0.018397
2024-03-29,-0.012118
2024-03-30,-0.003548
2024-03-31,0.023953


### Direction, 1 if return is positive, 0 if return is negative

In [3]:
direction = pd.DataFrame(
    {"Direction": btc_return["Returns"].apply(lambda x: 1 if x > 0 else 0)}
)
direction

,Direction
2014-09-18,0
2014-09-19,0
2014-09-20,1
2014-09-21,0
2014-09-22,1
...,...
2024-03-28,1
2024-03-29,0
2024-03-30,0
2024-03-31,1


In [4]:
direction.index.name = "date_dt"
direction.to_csv("../Data/BTC_direction.csv")

### Magnitude, 1 - 10, based on twitter paper

In [5]:
btc = yf.Ticker("BTC-USD").history(period="max")
btc["price_chg"] = btc["Close"].diff(1)
btc.dropna(inplace=True)

In [6]:
btc.index = pd.to_datetime(btc.index)
btc.index = btc.index.date
btc["price_chg"]

2014-09-18     -32.894012
2014-09-19     -29.644012
2014-09-20      14.108002
2014-09-21     -10.082977
2014-09-22       3.330994
                 ...     
2024-03-31    1688.343750
2024-04-01   -1631.500000
2024-04-02   -4255.175781
2024-04-03     533.839844
2024-04-04    2030.664062
Name: price_chg, Length: 3487, dtype: float64

In [7]:
bin_edges = [
    -float("inf"),
    -1320,
    -990,
    -660,
    -330,
    0,
    330,
    660,
    990,
    1320,
    float("inf"),
]

bin_labels = list(range(1, 11))

magnitude_df = pd.DataFrame(
    {"magnitude": pd.cut(btc["price_chg"], bins=bin_edges, labels=bin_labels)}
)

magnitude_df["magnitude"] = magnitude_df["magnitude"].astype(int)
magnitude_df.index.name = "date_dt"

magnitude_df

,magnitude
date_dt,
2014-09-18,5
2014-09-19,5
2014-09-20,6
2014-09-21,5
2014-09-22,6
...,...
2024-03-31,10
2024-04-01,1
2024-04-02,1


In [8]:
magnitude_df.to_csv("../Data/BTC_magnitude.csv")